In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
##### DATABASE CONNECTION ####
#####Load data function ####
def saveToSQL(dataframe, tablename, todo):
    from sqlalchemy import create_engine
    if 'source' in dataframe.columns:
        dataframe = dataframe.drop(columns=['source'])
    if 'cik' in dataframe.columns:
        dataframe = dataframe.drop(columns=['cik'])
    else:
        dataframe = dataframe
    engine = create_engine('########### HIDDEN CODE #################')
    dataframe.index = dataframe.index + 1
    try:
        dataframe.to_sql(tablename, con=engine, schema='mysql_portfolio', if_exists=todo, index=True, index_label='id', method=None)
        # print(f"data saved in {tablename} successfully.")
    except Exception as e:
        print(f"Error creating table or inserting data: {str(e)}")

print('Data connection successful!')

In [ ]:
import json
import csv
import urllib.request
import pandas as pd
import numpy as np
import pymysql
from io import BufferedReader
import mysql.connector as connection
from datetime import date



pymysql.converters.encoders[np.float64] = pymysql.converters.escape_float
pymysql.converters.conversions = pymysql.converters.encoders.copy()
pymysql.converters.conversions.update(pymysql.converters.decoders)

In [ ]:
######### DROP EXISTING BASE TABLES ##########
# import required data from mydql/database
import mysql.connector as connection
import pandas as pd

try:
    mydb = connection.connect('########### HIDDEN CODE #################')
    sql_statements = [
        "DROP TABLE IF EXISTS mysql_portfolio.symbol_list;",
         
        
        ########### HIDDEN CODE #################
    ]
    
    # Execute each SQL statement
    for sql in sql_statements:
        mycursor = mydb.cursor()
        mycursor.execute(sql)
        print(f"Executed: {sql}")
        mycursor.close()
        
except Exception as e:
    mydb.close()
    # print(str(e))

In [ ]:
#### LOAD all symbols 

#### METHOD 2 TO READ DATA 
api_key = "########### HIDDEN CODE #################"
df_symbol_list = pd.read_json(api_key)
df_symbol_list['date_added']= date.today()
# df_symbol_list.head(10)
saveToSQL(df_symbol_list,"symbol_list",'replace')
print('Entire Data loaded in table')
# call_proc('insert_datetime_income_statemnent')

In [ ]:
from datetime import date, timedelta

today = date.today()
first_day_of_current_month = date(today.year, today.month, 1)
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# print(first_day_of_current_month - timedelta(days=1))

last_month_end_string = last_day_of_previous_month.strftime("%Y-%m-%d")
last_month_end_string

In [ ]:
# Stock screener
# UPDATE DAILY
import urllib.request, json
import pandas as pd

exchanges = ["NYSE", "NASDAQ", "NSE"]

for exchange in exchanges:
    api_key = '########### HIDDEN CODE #################'
    df_stock_screener_usa = pd.read_json(api_key)
    df_stock_screener_usa['date_added']= date.today()
    saveToSQL(df_stock_screener_usa,"stock_screener",'append')
    print('Entire Data loaded in table for '+exchange)
#     df_income_statement.append(df_income_statement)
#     call_api(newurl)

In [ ]:
# Sector p/e 

import urllib.request, json
import pandas as pd

exchanges = ["NYSE", "NASDAQ", "NSE"]

for exchange in exchanges:
    api_key = '####### HIDDEN CODE ############'
    # print(api_key)
    df_sectorpe_usa = pd.read_json(api_key)
    df_sectorpe_usa['date_added']= date.today()
    saveToSQL(df_sectorpe_usa,"sector_pe",'append')
    print('Entire Data loaded in table for '+exchange)

In [ ]:
# Industry p/e
# UPDATE BY PROVIDING MANUAL DATE
import urllib.request, json
import pandas as pd

exchanges = ["NYSE", "NASDAQ", "NSE"]

for exchange in exchanges:
    api_key = '############### HIDDEN CODE ################'
    df_industrype_usa = pd.read_json(api_key)
    df_industrype_usa['date_added']= date.today()
    saveToSQL(df_industrype_usa,"industry_pe",'append')
    print('Entire Data loaded in table for '+exchange)
#     df_income_statement.append(df_income_statement)


In [ ]:
######### CHANGE EXCHANGE FOR PER SYMBOL ##########
# import required data from mydql/database
import mysql.connector as connection
import pandas as pd

try:
    mydb = connection.connect(####### HIDDEN CODE ############)
    query = "SELECT * FROM mysql_portfolio.symbol_list WHERE exchangeShortName in ('NASDAQ') AND type = 'stock';"
    df_symbol_list_sql = pd.read_sql(query,mydb)
#     df_symbol_list_sql.head(10)
    symbol = df_symbol_list_sql['symbol']
    
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

In [ ]:
df_symbol_list_sql

In [ ]:
import time
import pymysql
import requests

def connect_with_retry_1(api_url): 
    max_retries = 100
    retry_delay = 65  # seconds
    retries = 0
    

    while retries < max_retries:
        try:
            response = requests.get(api_url)
            # print(response)
            response.raise_for_status()
            
            # if response.status_code == 429:
            #     print(response.status_code)
            #     make_http_request(api_url,response)
            # else:
            # print("try code execution")
            read_json = pd.read_json(api_url)
            # read_json.raise_for_status()
            return read_json
        
        except pymysql.err.OperationalError as e:
            if e.args[0] == 1040:  # Too many connections error
                print("Too many connections. Retrying in {} seconds...".format(retry_delay))
                time.sleep(retry_delay)
                retries += 1
            else:
                raise  # Re-raise other OperationalError exceptions
        
        except requests.HTTPError as e:
            if response.status_code == 429:
                retry_after = int(response.headers.get('Retry-After', 0))
                print("Wait time:", retry_after)
                time.sleep(retry_after)
                retries += 1
                return connect_with_retry(api_url)
            else:
                print("Error:", e)
    # If max_retries is reached without a successful connection, handle the failure
    print("Failed to establish a database connection after {} retries.".format(max_retries))
    return None

In [ ]:
# Code to load shares float  - 1
import urllib.request, json
import pandas as pd

api_key = '######## HIDDEN CODE ##########'
symbol = df_symbol_list_sql['symbol']

for x in symbol:
    newurl = api_key.replace('stock_name',x)
    api_data = connect_with_retry(newurl)
    df_share_float = api_data
    df_share_float['date_added']= date.today()
    saveToSQL(df_share_float,"shares_float",'append')
#     df_income_statement.append(df_income_statement)
#     call_api(newurl)
print('Entire Data loaded in table')

index_sql = "CREATE INDEX idx_symbol ON mysql_portfolio.shares_float (symbol(255));"

constraint_sql = """
ALTER TABLE mysql_portfolio.shares_float
ADD CONSTRAINT unique_shares_float UNIQUE (symbol,freeFloat,floatShares);
"""
# Database connection
engine = create_engine('####### HIDDEN CODE ############')

# Execute SQL statements
with engine.connect() as connection:
    connection.execute(index_sql)
    connection.execute(constraint_sql)

In [ ]:
# Historical Prices
# UPDATE BY PROVIDING MANUAL DATE
import urllib.request, json
import pandas as pd

# symbol = ['AAPL','MSFT','F','AMD','FB','MRF.NS','SHAKTIPUMP.NS','OAL.NS','KSCL.NS','MARKSANS.NS','LICHSGFIN.NS','CANFINHOME.NS','SASKEN.NS','HINDPETRO.NS','TIDEWATER.NS','HIL.NS','BALKRISIND.NS','NAVINFLUOR.NS']
api_key = '####### HIDDEN CODE ##########'
symbol = df_symbol_list_sql['symbol']
idx = 0
for x in symbol:
    try:
#     print(symbol)
        newurl = api_key.replace('symbol',x)
#         print(x)
        with urllib.request.urlopen(newurl) as newurl:
            data = json.loads(newurl.read().decode())
#           print(data)
            df_hist_price = pd.json_normalize(data['historical'])
#           df_hist_price ['symbol'] = x   # This will add a new column to dataframe at last position by default
            df_hist_price.insert(loc=idx, column='symbol', value=x) # This will add a new column at specific/required location
#           dt = pd.DataFrame(data =[{'symbol':x,}])
#           df_hist_price.head(10)
            df_hist_price['date_added']= date.today()
            saveToSQL(df_hist_price,"historical_prices",'append')
    except Exception as val:
        print(val)
        continue
print('Entire data is loaded')

index_sql = "CREATE INDEX idx_symbol ON mysql_portfolio.historical_prices (symbol(255));"

constraint_sql = """
ALTER TABLE mysql_portfolio.historical_prices
ADD CONSTRAINT unique_historical_prices UNIQUE (symbol,open,adjClose);
"""
# Database connection
engine = create_engine('####### HIDDEN CODE ############')

# Execute SQL statements
with engine.connect() as connection:
    connection.execute(index_sql)
    connection.execute(constraint_sql)

In [ ]:
# Code to load income statement  - 1
import urllib.request, json
import pandas as pd

api_key = '##### HIDDEN CODE #####'
symbol = df_symbol_list_sql['symbol']
for x in symbol:
    try:
        newurl = api_key.replace('symbol',x)
        api_data = connect_with_retry(newurl)
        df_income_statement = api_data
        df_income_statement = df_income_statement.replace(r'^\s*$', np.NaN, regex=True) # This replaces special characters/Null/blank values with NaN
        # df_income_statement['cik'] = df_income_statement['cik'].fillna(0) # This replaces NaN with 0
#         df_income_statement
        df_income_statement['date_added']= date.today()
        saveToSQL(df_income_statement,"income_statement",'append')
    except Exception as val:
        print(val)
        continue
#     df_income_statement.append(df_income_statement)
#     call_api(newurl)
print('Entire Data loaded in table')

# Adding index to table 
index_sql = "CREATE INDEX idx_symbol ON mysql_portfolio.income_statement (symbol(255));"

# Adding unique constraint
constraint_sql = """
ALTER TABLE mysql_portfolio.income_statement
ADD CONSTRAINT unique_income_statement UNIQUE (symbol,reportedCurrency,calendarYear,revenue);
"""
# Database connection
engine = create_engine('####### HIDDEN CODE ############')

# Execute SQL statements
with engine.connect() as connection:
    connection.execute(index_sql)
    connection.execute(constraint_sql)

In [ ]:
# Code to load balance sheet
import urllib.request, json
import pandas as pd

rowcount = 0
api_key = '####### HIDDEN CODE ############'
symbol = df_symbol_list_sql['symbol']

for x in symbol:
    try:
        newurl = api_key.replace('symbol', x)
        api_data = connect_with_retry(newurl)
        df_balance_sheet = api_data

        # Check if the DataFrame is empty
        if df_balance_sheet.empty:
            print(f"Warning: No data found for symbol {x}. Skipping.")
            continue

        df_balance_sheet = df_balance_sheet.replace(r'^\s*$', np.NaN, regex=True)
        # df_balance_sheet['cik'] = df_balance_sheet['cik'].fillna(0)
        
        df_balance_sheet['date_added'] = date.today()
        saveToSQL(df_balance_sheet, "balance_sheet", 'append')
    except Exception as val:
        print(val)
        continue

print('Entire Data loaded in table')

# Adding index to table 
index_sql = "CREATE INDEX idx_symbol ON mysql_portfolio.balance_sheet (symbol(255));"

# Adding unique constraint
constraint_sql = """
ALTER TABLE mysql_portfolio.balance_sheet
ADD CONSTRAINT unique_balance_sheet UNIQUE (symbol,reportedCurrency,calendarYear,cashAndCashEquivalents);
"""
# Database connection
engine = create_engine('####### HIDDEN CODE ############')

# Execute SQL statements
with engine.connect() as connection:
    connection.execute(index_sql)
    connection.execute(constraint_sql)

In [ ]:
# Code to load cash flow statement
import urllib.request, json
import pandas as pd

rowcount = 0
####### HIDDEN CODE ############


